In [1]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from django.apps import apps as django_apps

from django.db.models.signals import post_save
post_save.disconnect(dispatch_uid='pre_flourish_assent_post_save')
post_save.disconnect(dispatch_uid='pre_flourish_child_dummy_consent_on_post_save')

False

In [2]:
def get_childconsent_data(qs_data, parent_consent):
    fields = ['first_name', 'last_name', 'gender', 'child_dob', 'child_test', 'child_remain_in_study',
              'child_preg_test', 'child_knows_status', 'identity', 'identity_type', 'confirm_identity',
              'future_studies_contact', 'consent_datetime']
    data = {'subject_consent': parent_consent}
    for field in fields:
        data[field] = qs_data.get(field)
    return data

In [3]:
def remove_related_objs(child_identifier):
    registeredsubject = PreFlourishRegisteredSubject.objects.filter(
        subject_identifier=child_identifier)
    identifiermodel = IdentifierModel.objects.filter(identifier=child_identifier)
    registeredsubject.delete()
    identifiermodel.delete()

In [4]:
siblings = []
consents = PreFlourishConsent.objects.all()
for consent in consents:
    if consent.preflourishcaregiverchildconsent_set.count() > 1:
        siblings.append(consent)

In [5]:
idxs_to_change = {}

for consent in siblings:
    idxs = []
    child_consents = consent.preflourishcaregiverchildconsent_set.order_by(
        'consent_datetime').values_list('subject_identifier', flat=True)
    for child_sidx in child_consents:
        if child_sidx[-3:] in ['-20', '-30', '-40']:
            idxs.append(child_sidx)

    idxs_to_change.update({f'{consent.subject_identifier}': idxs})
    pf_childconsents = consent.preflourishcaregiverchildconsent_set.filter(
        subject_identifier__in=idxs)
    pf_childconsents_copy = []
    for childconsent in pf_childconsents:
        dict_copy = childconsent.__dict__.copy()
        pf_childconsents_copy.append(get_childconsent_data(dict_copy, consent))
        remove_related_objs(childconsent.subject_identifier)
    pf_childconsents.delete()

    for obj in pf_childconsents_copy:
        # Recreate objects
        childconsent = PreFlourishCaregiverChildConsent(**obj)
        childconsent.save()

In [6]:
idxs_to_change

{'B142-040991222-6P': ['B142-040991222-6P-20'],
 'B142-040991228-3P': ['B142-040991228-3P-20'],
 'B142-040991242-4P': ['B142-040991242-4P-20', 'B142-040991242-4P-30'],
 'B142-040991244-0P': ['B142-040991244-0P-20']}

In [11]:
from pre_flourish.models.appointment import Appointment

# forms to change: Child Assent, DummyConsent, OnSchedules, Appointments, Visits, Contacts, OffSchedule, OffStudy
for caregiver_idx, child_idxs in idxs_to_change.items():
    for idx in child_idxs:
        related_chilconsent = None
        try:
            assent = PreFlourishChildAssent.objects.get(subject_identifier=idx)
        except PreFlourishChildAssent.DoesNotExist:
            assent = None

        if assent and assent.identity:
            related_chilconsent = PreFlourishCaregiverChildConsent.objects.get(
                identity=assent.identity)
        elif assent:
            related_chilconsent = PreFlourishCaregiverChildConsent.objects.get(
                first_name=assent.first_name, last_name=assent.last_name, child_dob=assent.dob)

        new_child_identifier = getattr(related_chilconsent, 'subject_identifier', None)
        if not new_child_identifier:
            print(idx)
        if new_child_identifier:
            print(idx)
            post_save.disconnect(dispatch_uid='pre_flourish_assent_post_save')
            post_save.disconnect(dispatch_uid='pre_flourish_child_dummy_consent_on_post_save')

            OnScheduleChildPreFlourish.objects.filter(subject_identifier=idx).update(
                subject_identifier=new_child_identifier)

            SubjectScheduleHistory.objects.filter(subject_identifier=idx).update(
                subject_identifier=new_child_identifier)

            Appointment.objects.filter(subject_identifier=idx).update(
                subject_identifier=new_child_identifier)

            PreFlourishVisit.objects.filter(subject_identifier=idx).update(
                subject_identifier=new_child_identifier)
            # Update metadata
            CrfMetadata.objects.filter(subject_identifier=idx).update(
                subject_identifier=new_child_identifier)
            RequisitionMetadata.objects.filter(subject_identifier=idx).update(
                subject_identifier=new_child_identifier)

            PreFlourishContact.objects.filter(subject_identifier=idx).update(
                subject_identifier=new_child_identifier)

            offstudy = PreFlourishChildOffStudy.objects.filter(subject_identifier=idx)
            if offstudy:
                offstudy.update(subject_identifier=new_child_identifier)
                ChildOffSchedule.objects.filter(
                    subject_identifier=idx).update(subject_identifier=new_child_identifier)

            PreFlourishChildDummySubjectConsent.objects.filter(subject_identifier=idx).update(
                subject_identifier=new_child_identifier)

            assent.subject_identifier = new_child_identifier
            assent.save()

B142-040991222-6P-20
B142-040991228-3P-20
B142-040991242-4P-20
B142-040991242-4P-30
B142-040991244-0P-20
